# 05 - Export Dashboard Data

Generates pre-processed JSON files for the React dashboard. Replaces mock data with real Transfermarkt results.

In [ ]:

import pandas as pd
import numpy as np
import sqlite3
import json
from pathlib import Path

DB_PATH = Path('..') / 'data' / 'processed' / 'football.db'
OUT_DIR = Path('..') / 'dashboard' / 'public' / 'data'

conn = sqlite3.connect(DB_PATH)
LEAGUE_NAMES = {'GB1': 'Premier League', 'ES1': 'La Liga', 'IT1': 'Serie A', 'L1': 'Bundesliga', 'FR1': 'Ligue 1'}

def save(name, data):
    path = OUT_DIR / name
    with open(path, 'w') as f:
        json.dump(data, f, indent=2, default=str)
    n = len(data) if isinstance(data, list) else sum(len(v) if isinstance(v, list) else 1 for v in data.values())
    print(f"  {name} -> {path.stat().st_size/1024:.1f} KB ({n} records)")

print(f"Output: {OUT_DIR}")


## 1. market_overview.json

In [ ]:

df = pd.read_sql('''
    WITH yearly AS (
        SELECT
            CAST(strftime('%Y', date) AS INTEGER) as year,
            player_club_domestic_competition_id as league_id,
            SUM(market_value_in_eur) as total_market_value,
            COUNT(DISTINCT player_id) as player_count,
            AVG(market_value_in_eur) as avg_player_value
        FROM player_valuations
        WHERE player_club_domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
          AND market_value_in_eur > 0
          AND strftime('%m', date) = '01'
        GROUP BY year, league_id
    )
    SELECT year, league_id, total_market_value, player_count, avg_player_value,
        ROUND(
            (total_market_value - LAG(total_market_value) OVER (PARTITION BY league_id ORDER BY year))
            * 100.0
            / NULLIF(LAG(total_market_value) OVER (PARTITION BY league_id ORDER BY year), 0),
        1) as yoy_growth_pct
    FROM yearly
    WHERE year BETWEEN 2012 AND 2025
    ORDER BY year, league_id
''', conn)
df['league_name'] = df['league_id'].map(LEAGUE_NAMES)
df['total_market_value'] = df['total_market_value'].round(0)
df['avg_player_value'] = df['avg_player_value'].round(0)
df['yoy_growth_pct'] = df['yoy_growth_pct'].where(df['yoy_growth_pct'].notna(), other=None)
save('market_overview.json', df.to_dict('records'))
display(df[df['league_id']=='GB1'][['year','total_market_value','player_count','yoy_growth_pct']].tail(5))


## 2. league_comparison.json

In [ ]:

df = pd.read_sql('''
    WITH latest_year AS (
        SELECT MAX(CAST(strftime('%Y', date) AS INTEGER)) - 1 as yr
        FROM player_valuations
        WHERE player_club_domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
    ),
    league_totals AS (
        SELECT pv.player_club_domestic_competition_id as league_id,
            SUM(pv.market_value_in_eur) as total_value,
            AVG(pv.market_value_in_eur) as avg_value,
            COUNT(DISTINCT pv.player_id) as player_count
        FROM player_valuations pv, latest_year ly
        WHERE pv.player_club_domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
          AND pv.market_value_in_eur > 0
          AND CAST(strftime('%Y', pv.date) AS INTEGER) = ly.yr
        GROUP BY pv.player_club_domestic_competition_id
    ),
    club_vals AS (
        SELECT c.domestic_competition_id as league_id, c.name as club_name,
            c.total_market_value,
            ROW_NUMBER() OVER (PARTITION BY c.domestic_competition_id ORDER BY c.total_market_value DESC) as rn
        FROM clubs c
        WHERE c.domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
          AND c.total_market_value > 0
    )
    SELECT lt.league_id, lt.total_value, lt.avg_value, lt.player_count,
        cv.club_name as top_club, cv.total_market_value as top_club_value
    FROM league_totals lt
    LEFT JOIN club_vals cv ON lt.league_id = cv.league_id AND cv.rn = 1
    ORDER BY lt.total_value DESC
''', conn)
df['league'] = df['league_id'].map(LEAGUE_NAMES)
df[['total_value','avg_value','top_club_value']] = df[['total_value','avg_value','top_club_value']].round(0)
save('league_comparison.json', df.to_dict('records'))
display(df[['league','player_count','top_club']].round(0))


## 3. top_transfers.json

In [ ]:

df = pd.read_sql('''
    SELECT player_name, from_club_name as from_club, to_club_name as to_club,
        ROUND(transfer_fee) as fee, transfer_season as season, transfer_date
    FROM transfers
    WHERE transfer_fee > 0
    ORDER BY transfer_fee DESC LIMIT 15
''', conn)
save('top_transfers.json', df.to_dict('records'))
display(df[['player_name','from_club','to_club','fee']].head(10))


## 4. age_curves.json

In [ ]:

df = pd.read_sql('''
    SELECT
        CAST((julianday(pv.date) - julianday(p.date_of_birth)) / 365.25 AS INTEGER) as age,
        p.position, pv.market_value_in_eur / 1e6 as value_m
    FROM player_valuations pv
    JOIN players p ON pv.player_id = p.player_id
    WHERE p.position IN ('Attack', 'Midfield', 'Defender', 'Goalkeeper')
      AND p.date_of_birth IS NOT NULL
      AND pv.market_value_in_eur > 0
      AND pv.date >= '2015-01-01'
''', conn)
df = df[(df['age'] >= 17) & (df['age'] <= 38)]
pivot = df.groupby(['age','position'])['value_m'].median().unstack('position').reset_index()
for col in ['Attack','Midfield','Defender','Goalkeeper']:
    if col in pivot.columns:
        pivot[col] = pivot[col].round(3)
save('age_curves.json', pivot.to_dict('records'))
display(pivot.head(5))


## 5. risk_metrics.json

In [ ]:

df_vol = pd.read_sql('''
    WITH monthly AS (
        SELECT player_club_domestic_competition_id as league_id,
            strftime('%Y', date) as season, strftime('%m', date) as month,
            AVG(market_value_in_eur) as avg_val
        FROM player_valuations
        WHERE player_club_domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
          AND market_value_in_eur > 0 AND date >= '2015-01-01'
        GROUP BY league_id, season, month
    ),
    returns AS (
        SELECT league_id, season,
            (avg_val - LAG(avg_val) OVER (PARTITION BY league_id ORDER BY season, month))
            * 100.0 / NULLIF(LAG(avg_val) OVER (PARTITION BY league_id ORDER BY season, month), 0)
            as monthly_ret
        FROM monthly
    )
    SELECT league_id, season,
        ROUND(SQRT(
            (SUM(monthly_ret*monthly_ret)/COUNT(*)) - (AVG(monthly_ret)*AVG(monthly_ret))
        ), 2) as volatility
    FROM returns
    WHERE monthly_ret IS NOT NULL
    GROUP BY league_id, season HAVING COUNT(*) >= 6
    ORDER BY season, league_id
''', conn)
df_vol['league'] = df_vol['league_id'].map(LEAGUE_NAMES)
df_vol = df_vol[['league','season','volatility']].dropna()

df_dd = pd.read_sql('''
    WITH player_peak AS (
        SELECT pv.player_id, p.position, pv.market_value_in_eur,
            MAX(pv.market_value_in_eur) OVER (
                PARTITION BY pv.player_id ORDER BY pv.date
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) as running_max
        FROM player_valuations pv
        JOIN players p ON pv.player_id = p.player_id
        WHERE p.position IN ('Attack', 'Midfield', 'Defender', 'Goalkeeper')
          AND pv.market_value_in_eur > 0
    ),
    drawdown AS (
        SELECT player_id, position,
            ROUND((market_value_in_eur - running_max) * 100.0 / running_max, 2) as drawdown_pct
        FROM player_peak WHERE running_max > 0
    ),
    player_max_dd AS (
        SELECT player_id, position, MIN(drawdown_pct) as max_drawdown
        FROM drawdown GROUP BY player_id, position
    )
    SELECT position,
        ROUND(AVG(max_drawdown), 1) as avg_max_drawdown,
        ROUND(MIN(max_drawdown), 1) as worst_drawdown,
        COUNT(*) as player_count
    FROM player_max_dd GROUP BY position ORDER BY avg_max_drawdown
''', conn)

df_dep = pd.read_sql('''
    WITH age_brackets AS (
        SELECT p.position,
            CASE
                WHEN CAST((julianday(pv.date) - julianday(p.date_of_birth)) / 365.25 AS INT) < 21 THEN 'U21'
                WHEN CAST((julianday(pv.date) - julianday(p.date_of_birth)) / 365.25 AS INT) < 25 THEN '21-24'
                WHEN CAST((julianday(pv.date) - julianday(p.date_of_birth)) / 365.25 AS INT) < 28 THEN '25-27'
                WHEN CAST((julianday(pv.date) - julianday(p.date_of_birth)) / 365.25 AS INT) < 31 THEN '28-30'
                ELSE '31+'
            END as age_bracket,
            (pv.market_value_in_eur - LAG(pv.market_value_in_eur) OVER (PARTITION BY pv.player_id ORDER BY pv.date))
            * 100.0 / NULLIF(LAG(pv.market_value_in_eur) OVER (PARTITION BY pv.player_id ORDER BY pv.date), 0)
            as change_pct
        FROM player_valuations pv
        JOIN players p ON pv.player_id = p.player_id
        WHERE p.position IN ('Attack', 'Midfield', 'Defender', 'Goalkeeper')
          AND p.date_of_birth IS NOT NULL AND pv.market_value_in_eur > 0
    )
    SELECT position, age_bracket,
        ROUND(AVG(change_pct), 2) as avg_change_pct,
        ROUND(AVG(CASE WHEN change_pct < 0 THEN change_pct ELSE NULL END), 2) as depreciation_rate
    FROM age_brackets
    WHERE change_pct IS NOT NULL AND ABS(change_pct) < 200
    GROUP BY position, age_bracket ORDER BY position, age_bracket
''', conn)

df_sharpe = pd.read_sql('''
    WITH monthly_vals AS (
        SELECT player_club_domestic_competition_id as league_id,
            strftime('%Y-%m', date) as month, AVG(market_value_in_eur) as avg_val
        FROM player_valuations
        WHERE player_club_domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
          AND market_value_in_eur > 0 AND date >= '2015-01-01'
        GROUP BY league_id, month
    ),
    monthly_rets AS (
        SELECT league_id, month,
            (avg_val - LAG(avg_val) OVER (PARTITION BY league_id ORDER BY month))
            * 100.0 / NULLIF(LAG(avg_val) OVER (PARTITION BY league_id ORDER BY month), 0)
            as ret
        FROM monthly_vals
    )
    SELECT league_id,
        ROUND(AVG(ret) * 12, 2) as avg_return,
        ROUND(SQRT((SUM(ret*ret)/COUNT(*)) - (AVG(ret)*AVG(ret))) * SQRT(12), 2) as volatility,
        ROUND(AVG(ret) / NULLIF(SQRT((SUM(ret*ret)/COUNT(*)) - (AVG(ret)*AVG(ret))), 0), 3) as sharpe_ratio
    FROM monthly_rets
    WHERE ret IS NOT NULL GROUP BY league_id ORDER BY sharpe_ratio DESC
''', conn)
df_sharpe['league'] = df_sharpe['league_id'].map(LEAGUE_NAMES)

risk = {
    'volatility_heatmap': df_vol.to_dict('records'),
    'drawdown_by_position': df_dd.to_dict('records'),
    'depreciation_rates': df_dep.to_dict('records'),
    'sharpe_ratios': df_sharpe[['league','sharpe_ratio','avg_return','volatility']].to_dict('records'),
}
save('risk_metrics.json', risk)
print("\nDrawdown:")
display(df_dd)
print("\nSharpe:")
display(df_sharpe[['league','sharpe_ratio','avg_return','volatility']])


In [ ]:
# Don't close yet -- more exports below

## 6. transfer_analytics.json

ROI distribution, club net spend, and fee-vs-value-change scatter. Uses a correlated subquery to find market value 1 year after each transfer.

In [ ]:
# Base query: transfers with market value at transfer time and ~1 year later
# Correlated subquery is slow (~5 min) so we do it once and reuse
# Note: SQLite bug limits outer-column references per subquery, so we use
# a simpler approach: first valuation after +10 months (ORDER BY date ASC LIMIT 1)
print("Running correlated subquery (this takes a few minutes)...")
df_transfers = pd.read_sql('''
    SELECT
        t.player_name,
        t.transfer_fee,
        t.transfer_date,
        t.to_club_name,
        t.from_club_name,
        p.date_of_birth,
        p.position,
        (SELECT pv.market_value_in_eur
         FROM player_valuations pv
         WHERE pv.player_id = t.player_id
           AND pv.date <= t.transfer_date
         ORDER BY pv.date DESC LIMIT 1
        ) AS value_at_transfer,
        (SELECT pv2.market_value_in_eur
         FROM player_valuations pv2
         WHERE pv2.player_id = t.player_id
           AND pv2.date > DATE(t.transfer_date, '+10 months')
         ORDER BY pv2.date ASC LIMIT 1
        ) AS value_after_1yr
    FROM transfers t
    JOIN players p ON t.player_id = p.player_id
    WHERE t.transfer_fee > 5000000
      AND t.transfer_date >= '2015-01-01'
      AND t.transfer_date <= '2024-01-01'
''', conn)

# Calculate ROI and value change
df_transfers['roi_pct'] = (
    (df_transfers['value_after_1yr'] - df_transfers['transfer_fee'])
    / df_transfers['transfer_fee'] * 100
).round(1)

df_transfers['value_change_pct'] = (
    (df_transfers['value_after_1yr'] - df_transfers['value_at_transfer'])
    / df_transfers['value_at_transfer'] * 100
).round(1)

# Age at transfer (handle NaN birth dates)
birth = pd.to_datetime(df_transfers['date_of_birth'], errors='coerce')
transfer = pd.to_datetime(df_transfers['transfer_date'], errors='coerce')
df_transfers['age_at_transfer'] = ((transfer - birth).dt.days / 365.25).round(0)

# Drop rows without 1-year valuation
df_valid = df_transfers.dropna(subset=['value_after_1yr', 'value_at_transfer'])
print(f"Transfers with fee >5M: {len(df_transfers)}, with 1yr valuation: {len(df_valid)}")

# --- ROI Distribution ---
def roi_category(roi):
    if roi > 50: return 'Excellent (>50%)'
    if roi >= 0: return 'Positive (0-50%)'
    if roi >= -50: return 'Moderate Loss'
    return 'Significant Loss'

df_valid = df_valid.copy()
df_valid['roi_cat'] = df_valid['roi_pct'].apply(roi_category)
roi_dist = df_valid.groupby('roi_cat').agg(
    count=('roi_pct', 'size'),
    avg_roi=('roi_pct', 'mean')
).round(1).reset_index().rename(columns={'roi_cat': 'category'})

# Sort in logical order
cat_order = ['Excellent (>50%)', 'Positive (0-50%)', 'Moderate Loss', 'Significant Loss']
roi_dist['_order'] = roi_dist['category'].map({c: i for i, c in enumerate(cat_order)})
roi_dist = roi_dist.sort_values('_order').drop(columns='_order')

# --- Net Spend (last 5 years) ---
df_paid = pd.read_sql('''
    SELECT to_club_name AS club, SUM(transfer_fee) AS paid
    FROM transfers
    WHERE transfer_fee > 0 AND transfer_date >= DATE('now', '-5 years')
    GROUP BY to_club_name
''', conn)

df_received = pd.read_sql('''
    SELECT from_club_name AS club, SUM(transfer_fee) AS received
    FROM transfers
    WHERE transfer_fee > 0 AND transfer_date >= DATE('now', '-5 years')
    GROUP BY from_club_name
''', conn)

df_net = df_paid.merge(df_received, on='club', how='outer').fillna(0)
df_net['net_spend'] = (df_net['received'] - df_net['paid']).astype(int)

# Top 6 spenders + Top 6 sellers
top_spenders = df_net.nsmallest(6, 'net_spend')
top_sellers = df_net.nlargest(6, 'net_spend')
net_spend = pd.concat([top_spenders, top_sellers.iloc[::-1]]).drop_duplicates(subset='club')
net_spend = net_spend[['club', 'net_spend']].sort_values('net_spend')

# --- Scatter: Top 200 transfers ---
scatter = df_valid.nlargest(200, 'transfer_fee')[['transfer_fee', 'value_change_pct', 'age_at_transfer', 'player_name']].copy()
scatter.columns = ['fee', 'value_change', 'age', 'name']
scatter = scatter.dropna()
scatter['age'] = scatter['age'].astype(int)

# --- Assemble and save ---
analytics = {
    'roi_distribution': roi_dist.to_dict('records'),
    'net_spend': net_spend.to_dict('records'),
    'scatter': scatter.to_dict('records'),
    'total_analyzed': int(len(df_valid)),
}
save('transfer_analytics.json', analytics)
display(roi_dist)
print(f"\nNet spend entries: {len(net_spend)}, Scatter points: {len(scatter)}")

## 7. club_financials.json

Top 20 clubs by squad value (Big 5 leagues). Reuses the SQL from `sql/07_club_financials.sql` and adds age group breakdowns.

In [ ]:
# Read the existing SQL file
sql_path = Path('..') / 'sql' / '07_club_financials.sql'
club_sql = sql_path.read_text(encoding='utf-8')

df_clubs = pd.read_sql(club_sql, conn)

# --- Fix 1: League names (competitions table has slugs like "premier-league") ---
# Map via domestic_competition_id from clubs table
league_for_club = pd.read_sql('''
    SELECT name AS club_name, domestic_competition_id AS league_id
    FROM clubs
    WHERE domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
''', conn)
club_league_map = dict(zip(league_for_club['club_name'], league_for_club['league_id'].map(LEAGUE_NAMES)))

# --- Fix 2: Shorten club names ---
# Remove common suffixes from full legal names
def shorten_club_name(name):
    replacements = [
        ' Football Club', ' Club de Fútbol', ' Fútbol Club',
        ' S.p.A.', ' S.A.D.', ' S.S.C.', ' S.p.a.',
        ' Associazione Calcica', ' Associazione Calcio',
        ' Associazione Sportiva',
    ]
    short = name
    for r in replacements:
        short = short.replace(r, '')
    # Specific well-known mappings for readability
    name_map = {
        'Real Madrid': 'Real Madrid',
        'Manchester City': 'Man City',
        'Arsenal': 'Arsenal',
        'Paris Saint-Germain': 'PSG',
        'Chelsea': 'Chelsea',
        'Liverpool': 'Liverpool',
        'Futbol Club Barcelona': 'Barcelona',
        'FC Bayern München': 'Bayern Munich',
        'Tottenham Hotspur': 'Tottenham',
        'Manchester United': 'Man United',
        'Newcastle United': 'Newcastle',
        'Football Club Internazionale Milano': 'Inter Milan',
        'Milan': 'AC Milan',
        'Aston Villa': 'Aston Villa',
        'Nottingham Forest': 'Nott. Forest',
        'Club Atlético de Madrid': 'Atletico Madrid',
        'Juventus': 'Juventus',
        'Crystal Palace': 'Crystal Palace',
        'Roma': 'AS Roma',
        'Brighton and Hove Albion': 'Brighton',
    }
    return name_map.get(short.strip(), short.strip())

# Age group breakdown per club
df_age = pd.read_sql('''
    SELECT
        cl.name AS club_name,
        CASE
            WHEN CAST((julianday('now') - julianday(p.date_of_birth)) / 365.25 AS INTEGER) < 21 THEN 'U21'
            WHEN CAST((julianday('now') - julianday(p.date_of_birth)) / 365.25 AS INTEGER) < 25 THEN '21-24'
            WHEN CAST((julianday('now') - julianday(p.date_of_birth)) / 365.25 AS INTEGER) < 29 THEN '25-28'
            WHEN CAST((julianday('now') - julianday(p.date_of_birth)) / 365.25 AS INTEGER) < 33 THEN '29-32'
            ELSE '33+'
        END AS age_group,
        COUNT(*) AS count,
        SUM(p.market_value_in_eur) AS value
    FROM players p
    JOIN clubs cl ON p.current_club_id = cl.club_id
    WHERE cl.domestic_competition_id IN ('GB1','ES1','IT1','L1','FR1')
      AND p.date_of_birth IS NOT NULL
      AND p.market_value_in_eur > 0
    GROUP BY cl.name, age_group
    ORDER BY cl.name, age_group
''', conn)

# --- Fix 3: Sort age groups logically ---
AGE_ORDER = ['U21', '21-24', '25-28', '29-32', '33+']

age_by_club = {}
for club_name, group in df_age.groupby('club_name'):
    records = group[['age_group', 'count', 'value']].rename(columns={'age_group': 'group'}).to_dict('records')
    records.sort(key=lambda r: AGE_ORDER.index(r['group']) if r['group'] in AGE_ORDER else 99)
    age_by_club[club_name] = records

# --- Fix 4: ROI -- cap at reasonable range, use net_transfer_balance for context ---
# Original formula: (squad_value - invested) / invested * 100
# Problem: clubs with low investment but high academy value get absurd ROI
# Solution: cap ROI and add net_balance as additional context

df_top = df_clubs.head(20)
clubs_out = []
for _, row in df_top.iterrows():
    raw_roi = float(row['investment_roi_pct'] or 0)
    # Cap ROI at +/- 200% to avoid misleading extremes
    capped_roi = max(-200.0, min(200.0, raw_roi))

    clubs_out.append({
        'name': shorten_club_name(row['club_name']),
        'league': club_league_map.get(row['club_name'], row['league_name']),
        'squad_value': int(row['total_squad_value'] or 0),
        'avg_age': float(row['average_age'] or 0),
        'star_dependency': float(row['star_dependency_pct'] or 0),
        'invested_5yr': int(row['invested_5yr'] or 0),
        'roi': round(capped_roi, 1),
        'positions': {
            'Attack': int(row['attack_value'] or 0),
            'Midfield': int(row['midfield_value'] or 0),
            'Defender': int(row['defender_value'] or 0),
            'Goalkeeper': int(row['goalkeeper_value'] or 0),
        },
        'age_groups': age_by_club.get(row['club_name'], []),
    })

save('club_financials.json', clubs_out)
print(f"\nTop 5 clubs:")
for c in clubs_out[:5]:
    print(f"  {c['name']:<20} {c['league']:<18} {c['squad_value']/1e9:.2f}B  ROI={c['roi']}%")

## 8. player_positions.json

Position treemap (total market value by sub_position) and value distribution histogram.

In [ ]:
# Treemap: total market value by sub_position
df_treemap = pd.read_sql('''
    SELECT
        sub_position AS name,
        position,
        SUM(market_value_in_eur) AS size
    FROM players
    WHERE market_value_in_eur > 0
      AND sub_position IS NOT NULL
      AND position IS NOT NULL
    GROUP BY sub_position, position
    ORDER BY size DESC
''', conn)

# Value distribution histogram
df_hist = pd.read_sql('''
    SELECT
        CASE
            WHEN market_value_in_eur < 1000000 THEN '<1M'
            WHEN market_value_in_eur < 5000000 THEN '1-5M'
            WHEN market_value_in_eur < 10000000 THEN '5-10M'
            WHEN market_value_in_eur < 20000000 THEN '10-20M'
            WHEN market_value_in_eur < 50000000 THEN '20-50M'
            WHEN market_value_in_eur < 100000000 THEN '50-100M'
            ELSE '>100M'
        END AS range,
        COUNT(*) AS count
    FROM players
    WHERE market_value_in_eur > 0
    GROUP BY range
    ORDER BY MIN(market_value_in_eur)
''', conn)

total_players = int(pd.read_sql('SELECT COUNT(*) as n FROM players WHERE market_value_in_eur > 0', conn).iloc[0, 0])

positions = {
    'treemap': df_treemap.to_dict('records'),
    'value_distribution': df_hist.to_dict('records'),
    'total_players': total_players,
}
save('player_positions.json', positions)
display(df_treemap.head(5))
print(f"\nHistogram bins: {len(df_hist)}, Total players: {total_players}")

In [ ]:
conn.close()
print()
print('All 8 JSON files exported. Dashboard now uses real Transfermarkt data.')